In [1]:
import pandas as pd
import os
import re  # 引入正则表达式库
from openai import OpenAI

# ================= 配置区域 =================

# 1. 获取当前工作目录 (兼容 Jupyter 和普通脚本)
try:
    CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    CURRENT_DIR = os.getcwd()

print(f"当前工作目录: {CURRENT_DIR}")

# 2. 输入文件名
INPUT_FILENAME = "qeustion-去除括号.xlsx"
INPUT_PATH = os.path.join(CURRENT_DIR, INPUT_FILENAME)

# 3. 输出文件名
OUTPUT_FILENAME = "question_expanded_result_v2.xlsx" # 改个名，防止覆盖
OUTPUT_PATH = os.path.join(CURRENT_DIR, OUTPUT_FILENAME)

# 4. 模型 API 配置
OPENAI_API_KEY = "EMPTY"
OPENAI_API_BASE = "http://192.168.6.117:19534/v1"
MODEL_NAME = "Qwen/Qwen3-32B"

# ===========================================

def clean_model_output(text):
    """
    清洗模型输出：
    1. 去除 <think>...</think> 及其内容
    2. 去除首尾空格
    """
    if not text:
        return ""
    
    # 使用正则表达式去除 <think> 标签包裹的内容 (flags=re.DOTALL 让 . 可以匹配换行符)
    cleaned_text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    
    # 去除可能残留的空行和首尾空格
    return cleaned_text.strip()

def get_model_response(client, prompt):
    """调用大模型获取回复"""
    try:
        messages = [{"role": "user", "content": prompt}]
        
        chat_response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            temperature=0.7, 
            max_tokens=2048 # 稍微调大一点，防止截断，但我们会清洗
        )
        raw_content = chat_response.choices[0].message.content
        
        # 立即进行清洗
        return clean_model_output(raw_content)
        
    except Exception as e:
        print(f"API调用出错: {e}")
        return "ERROR"

def main():
    # 1. 初始化客户端
    client = OpenAI(
        api_key=OPENAI_API_KEY,
        base_url=OPENAI_API_BASE,
    )

    # 2. 读取数据
    print(f"正在读取文件路径: {INPUT_PATH}")
    
    if not os.path.exists(INPUT_PATH):
        print("❌ 错误：文件不存在！")
        return

    try:
        df = pd.read_excel(INPUT_PATH)
        
        # 检查是否有 '问题' 这一列
        if '问题' not in df.columns:
            question_col_name = df.columns[1] 
        else:
            question_col_name = '问题'
            
    except Exception as e:
        print(f"❌ 读取Excel失败: {e}")
        return

    # 准备存储结果的列表
    expanded_scheme1 = [] # 方案1结果
    expanded_scheme2 = [] # 方案2结果

    total_rows = len(df)
    print(f"✅ 读取成功，共 {total_rows} 条数据，开始AI扩写处理...")

    # 3. 循环处理每一行
    for index, row in df.iterrows():
        original_question = str(row[question_col_name])
        
        if pd.isna(original_question) or original_question.strip() == "" or original_question == "nan":
            expanded_scheme1.append("")
            expanded_scheme2.append("")
            continue

        print(f"[{index + 1}/{total_rows}] 处理中: {original_question[:15]}...")

        # --- 方案 1: 单句长句扩写 (Prompt已修改) ---
        prompt_1 = f"""
你是一个专业的Prompt优化专家。请将以下【原问题】扩写成【一句话】。
要求：
1. 必须合成一个结构复杂的长句，包含背景、限定条件和核心提问。
2. 严禁分段，严禁使用句号分割成多句话。
3. 去除所有客套话，直接输出扩写后的那一句话。

原问题：{original_question}
"""
        ans1 = get_model_response(client, prompt_1)
        expanded_scheme1.append(ans1)

        # --- 方案 2: 预测潜在问题 (3-5条) ---
        prompt_2 = f"""
请分析以下问题的意图，并预测用户在问完这个问题后，可能紧接着会问的3到5个相关问题。
要求：
1. 直接列出问题，每行一个。
2. 不需要任何解释或开场白。

原问题：{original_question}
"""
        ans2 = get_model_response(client, prompt_2)
        expanded_scheme2.append(ans2)

    # 4. 将结果写入 DataFrame
    df['方案1_单句扩写'] = expanded_scheme1
    df['方案2_潜在追问'] = expanded_scheme2

    # 5. 导出为 Excel
    print(f"正在保存结果到: {OUTPUT_PATH}...")
    try:
        df.to_excel(OUTPUT_PATH, index=False)
        print("🎉 处理完成！请查看新生成的文件。")
    except PermissionError:
        print("❌ 保存失败：请检查目标Excel文件是否正在被打开，请关闭后重试。")

if __name__ == "__main__":
    main()

当前工作目录: d:\jupyter
正在读取文件路径: d:\jupyter\qeustion-去除括号.xlsx
✅ 读取成功，共 83 条数据，开始AI扩写处理...
[1/83] 处理中: 请以提纲形式生成中美科技的竞争...
[2/83] 处理中: 请根据最近全球恐怖主义事件推导...
[3/83] 处理中: 请用福卡风格推测未来黄金价格走...
[4/83] 处理中: 请梳理出关于世界右翼政党主要分...
[5/83] 处理中: 请梳理出世界重大交通枢纽的分布...
[6/83] 处理中: 请整理出世界各地发生的主要战争...
[7/83] 处理中: 当全球科技竞赛进入深水区，技术...
[8/83] 处理中: 请以“AI产业下一个十年何处去...
[9/83] 处理中: 会议会展业正面临VR等科技的重...
[10/83] 处理中: 我想写一篇关于可控核聚变的行业...
[11/83] 处理中: 我想了解，中国企业在AI硬件上...
[12/83] 处理中: 分析预判中美在脑机接口领域谁将...
[13/83] 处理中: 分析预判未来AI发展路线会出现...
[14/83] 处理中: 分析预判未来电商的发展模式是否...
[15/83] 处理中: 最近国家出台了哪些顶层规划和政...
[16/83] 处理中: 房地产、医疗等民生领域开放市场...
[17/83] 处理中: 气候变化对于中国未来农业经济会...
[18/83] 处理中: 情绪经济的主要理论依据有哪些？...
[19/83] 处理中: 未来中国乡镇在经济发展中的定位...
[20/83] 处理中: 现在居民消费普遍理性，那未来内...
[21/83] 处理中: 中国科技发展进程如何，需要精准...
[22/83] 处理中: 中国经济目前地理发展格局如何，...
[23/83] 处理中: 地方债务风险有哪些化解途径；...
[24/83] 处理中: 人口老龄化对于产业韧性有何影响...
[25/83] 处理中: 未来有哪些新手段化解消费障碍...
[26/83] 处理中: 请参考福卡文章风格及逻辑，探讨...
[27/83] 处理中: 请参考福卡文章风格及逻辑，针对...
[28/83] 处理中: 请用福卡文章的风格，输出一篇关...
[29/83] 处理中: 输出一篇福卡风格的预判美国未来